In [ ]:
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az
import pytensor.tensor as pt
import sys
sys.path.append('..')
import bayes_ordinal as bo

# Load data first
data = pd.read_csv("../Dataset/Trolley.csv", sep=';')
data = data.dropna()
feature_cols = ['action', 'intention', 'contact']
X = data[feature_cols].values
y = data['response'].values
y_zero_based = y - 1

print(f"Data loaded: {data.shape[0]} observations, {len(np.unique(y))} categories")


In [ ]:
def partial_odds_model_fixed(
    y, X, K=None, priors=None, model_name="partial_odds_fixed"
):
    """Partial odds model WITHOUT default initvals for log-likelihood computation."""
    
    if K is None:
        K = len(np.unique(y))
    
    priors = priors or {}
    mu_g = priors.get("gamma_mu", 0.0)
    sigma_g = priors.get("gamma_sigma", 0.5)  # More conservative
    mu_b = priors.get("beta_mu", 0.0)
    sigma_b = priors.get("beta_sigma", 0.5)  # More conservative

    with pm.Model(name=model_name) as model:
        # Use more conservative priors to avoid -inf
        gamma = pm.Normal("gamma", mu=mu_g, sigma=sigma_g, shape=K-1)
        beta = pm.Normal("beta", mu=mu_b, sigma=sigma_b, shape=X.shape[1])
        
        # Build η matrix
        η = X @ beta
        η = η[:, None] + gamma
        
        # Stick-breaking
        θ = pm.math.sigmoid(η)
        rem = pt.ones_like(θ[:, :1])
        ps = []
        for j in range(K-1):
            pj = rem * θ[:, j : j+1]
            ps.append(pj)
            rem = rem * (1 - θ[:, j : j+1])
        ps.append(rem)
        p = pt.concatenate(ps, axis=1)
        
        # Likelihood
        pm.Categorical("y_obs", p=p, observed=y)
        
        # NO DEFAULT INITVALS! This enables log-likelihood computation
    
    return model

# Test the fixed partial odds model
priors = {
    "gamma_mu": 0.0, "gamma_sigma": 0.5,
    "beta_mu": 0.0, "beta_sigma": 0.5
}

partial_odds_fixed = partial_odds_model_fixed(y_zero_based, X, priors=priors)
print("✓ Fixed partial odds model created")


In [ ]:
# Test sampling the fixed partial odds model
print("Testing partial odds model sampling...")
with partial_odds_fixed:
    idata_test = pm.sample(
        draws=100, tune=100, chains=2,
        init="jitter+adapt_diag",
        idata_kwargs={"log_likelihood": True},  # This should work now!
        random_seed=42
    )

print("✓ Partial odds model samples successfully with log-likelihood!")
print(f"Log-likelihood shape: {idata_test.log_likelihood.y_obs.shape}")

# Check if we can compute LOO
try:
    loo_test = az.loo(idata_test)
    print(f"✓ LOO computation successful: {loo_test.elpd_loo:.2f}")
except Exception as e:
    print(f"LOO failed: {e}")
